<h1>성능 개선</h1>


In [9]:
#초기 설정
#데이터 불러오기
import pandas as pd

data_path = './data/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

# weather이 4인경우는 이상치였기에 제거
train = train[train['weather']!=4]

# train데이터와 test데이터 합치기
all_data = pd.concat([train, test], ignore_index=True) #ignore_index를 통해서 원래 index를 무시하고 새롭게 index를 생성(train과 test를 합쳤기에)

# 날짜 처리
all_data['datetime'] = pd.to_datetime(all_data['datetime'])
all_data['date'] = all_data['datetime'].dt.date
all_data['year'] = all_data['datetime'].dt.year
all_data['month'] = all_data['datetime'].dt.month
all_data['hour'] = all_data['datetime'].dt.hour
all_data['weekday'] = all_data['datetime'].dt.weekday

# 불필요한 열 제거
drop_features = ['datetime', 'casual', 'registered', 'date', 'month', 'windspeed']
all_data = all_data.drop(columns=drop_features)

# 데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 데이터 정리
X_train = X_train.drop(['count'],axis=1)
X_test = X_test.drop(['count'],axis=1)
y = train['count']

#평가지표
import numpy as np
def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    return np.sqrt(np.mean((log_true - log_pred)**2))

<h4>1. 릿지 회귀 모델</h4>

In [10]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge_model = Ridge()


In [11]:
ridge_params = {'max_iter':[3000],'alpha':[0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000]}

rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)


In [12]:
# estimator: 릿지모델, param_grid: 값 목록, scoring: 평가지표, cv: 교차검증 횟수
gridsearch_ridge_model = GridSearchCV(estimator=ridge_model, param_grid=ridge_params, scoring=rmsle_scorer, cv=5)

In [14]:
log_y = np.log(y)
gridsearch_ridge_model.fit(X_train, log_y)
print(f'최적 하이퍼파라미터 : {gridsearch_ridge_model.best_params_}')


최적 하이퍼파라미터 : {'alpha': 0.1, 'max_iter': 3000}


In [16]:
pred = gridsearch_ridge_model.best_estimator_.predict(X_train)

print(f'릿지 회귀 모델의 RMSLE 값: {rmsle(log_y, pred, True):.4f}')


릿지 회귀 모델의 RMSLE 값: 1.0205


<h4>2. 라쏘 회귀 모델</h4>

In [17]:
from sklearn.linear_model import Lasso

lasso_model = Lasso()

lasso_alpha = 1/np.array([0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000])
lasso_params = {'max_iter':[3000],'alpha':lasso_alpha}

gridsearch_lasso_model = GridSearchCV(estimator=lasso_model, param_grid=lasso_params, scoring=rmsle_scorer, cv=5)

log_y = np.log(y)
gridsearch_lasso_model.fit(X_train, log_y)

print(f'최적 하이퍼파라미터 : {gridsearch_lasso_model.best_params_}')



최적 하이퍼파라미터 : {'alpha': 0.00125, 'max_iter': 3000}


In [18]:
pred = gridsearch_lasso_model.best_estimator_.predict(X_train)

print(f'라쏘 회귀 모델의 RMSLE 값: {rmsle(log_y, pred, True):.4f}')


라쏘 회귀 모델의 RMSLE 값: 1.0205


<h4>3. 랜덤 포레스트 회귀 모델</h4>

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()

rf_params = {'random_state':[42], 'n_estimators':[100,120,140]}
gridsearch_random_forest_model = GridSearchCV(estimator=rf_model, param_grid=rf_params, scoring=rmsle_scorer, cv=5)

log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train, log_y)

print(f'최적 하이퍼파라미터 : {gridsearch_random_forest_model.best_params_}')

pred = gridsearch_random_forest_model.best_estimator_.predict(X_train)

print(f'랜덤 포레스트 회귀 모델의 RMSLE 값: {rmsle(log_y, pred, True):.4f}')


최적 하이퍼파라미터 : {'n_estimators': 140, 'random_state': 42}
랜덤 포레스트 회귀 모델의 RMSLE 값: 0.1127
